<a href="https://colab.research.google.com/github/yesyell/DA_School_1/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EA%B3%BC%ED%95%99%20%EB%B0%A9%EB%B2%95%EB%A1%A0/%EC%B9%B4%EC%9A%B4%ED%8A%B8_%EA%B8%B0%EB%B0%98%EC%9D%98_%EB%AC%B8%EC%84%9C_%EC%9E%91%EC%97%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import gensim
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Bow (Bag of words) 개념

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

# 예제 문장
sentences = ["나는 고양이를 좋아한다.", "나는 개를 좋아한다."]

# CountVectorizer 객체 생성
vectorizer = CountVectorizer()

# 문장을 벡터로 변환
X = vectorizer.fit_transform(sentences)

# 결과 출력
vocabulary = vectorizer.get_feature_names_out()
vectors = X.toarray()

vocabulary, vectors

(array(['개를', '고양이를', '나는', '좋아한다'], dtype=object),
 array([[0, 1, 1, 1],
        [1, 0, 1, 1]]))

In [2]:
# 예제 문장 4개 추가
extended_sentences = [
    "나는 고양이를 좋아한다.",
    "나는 개를 좋아한다.",
    "고양이는 귀엽다.",
    "개는 충성스럽다.",
    "이것은 예제 문장이다."
]

# 문장을 벡터로 변환
X_extended = vectorizer.fit_transform(extended_sentences)

# 결과 출력
extended_vocabulary = vectorizer.get_feature_names_out()
extended_vectors = X_extended.toarray()

# 결과를 하나의 리스트로 합치기
combined_result = list(zip(extended_vocabulary, extended_vectors.sum(axis=0)))

combined_result

[('개는', 1),
 ('개를', 1),
 ('고양이는', 1),
 ('고양이를', 1),
 ('귀엽다', 1),
 ('나는', 2),
 ('문장이다', 1),
 ('예제', 1),
 ('이것은', 1),
 ('좋아한다', 2),
 ('충성스럽다', 1)]

# IMDB 리뷰 데이터

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# from tensorflow.keras.datasets import imdb

# (train_input, train_target), (test_input, test_target) = imdb.load_data(
#     num_words=500)

# print(train_input.shape, test_input.shape)

In [9]:
# 데이터 로드 (예시: IMDB 영화 리뷰 데이터)
data = pd.read_csv('/content/IMDB Dataset.csv')
reviews = data['review']
sentiments = data['sentiment']

In [10]:
# 데이터 전처리 (토큰화, 불용어 제거 등)
import re
from nltk.corpus import stopwords

def preprocess(text):
    # HTML 태그 제거
    text = re.sub(r'<.*?>', '', text)
    # 숫자와 특수문자 제거
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # 소문자 변환 및 토큰화
    words = text.lower().split()
    # 불용어 제거
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

# 리뷰 전처리
reviews = reviews.apply(preprocess)

In [12]:
reviews

0        one reviewers mentioned watching oz episode yo...
1        wonderful little production filming technique ...
2        thought wonderful way spend time hot summer we...
3        basically theres family little boy jake thinks...
4        petter matteis love time money visually stunni...
                               ...                        
49995    thought movie right good job wasnt creative or...
49996    bad plot bad dialogue bad acting idiotic direc...
49997    catholic taught parochial elementary schools n...
49998    im going disagree previous comment side maltin...
49999    one expects star trek movies high art fans exp...
Name: review, Length: 50000, dtype: object

## Bow

In [16]:
vectorizer_bow = CountVectorizer(max_features=5000)
X_bow = vectorizer_bow.fit_transform(reviews)

X_train, X_test, y_train, y_test = train_test_split(
    X_bow, sentiments, test_size=0.2, random_state=42)

model_bow = LogisticRegression()
model_bow.fit(X_train, y_train)

y_pred = model_bow.predict(X_test)
print("Bag of Words Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Bag of Words Accuracy: 0.874
              precision    recall  f1-score   support

    negative       0.88      0.87      0.87      4961
    positive       0.87      0.88      0.88      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



## TF-IDF

In [17]:
# TF-IDF 변환기를 사용하여 텍스트를 벡터로 변환
vectorizer = TfidfVectorizer(max_features=5000)

X = vectorizer.fit_transform(reviews)
y = sentiments

# 학습 및 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# 로지스틱 회귀 모델 학습
model = LogisticRegression()
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)
print("TF-IDF Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

TF-IDF Accuracy: 0.8867
              precision    recall  f1-score   support

    negative       0.90      0.87      0.88      4961
    positive       0.88      0.90      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



## Word Embedding (Word2Vec)

In [ ]:
# Word2Vec 모델 학습
model = gensim.models.Word2Vec(sentences=reviews, vector_size=100,
                               window=5, min_count=5, workers=4)

# 단어 벡터의 평균으로 리뷰를 벡터로 변환
def get_vector(review):
    vector = []
    for word in review:
        try:
            vector.append(model.wv[word])
        except KeyError:
            pass
    return np.mean(vector, axis=0)

X = reviews.apply(get_vector)
y = sentiments

# 학습 및 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# 로지스틱 회귀 모델 학습
model = LogisticRegression()
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

IMDB 영화 리뷰 데이터셋을 이용한 감성 분석에 대한 세 가지 텍스트 표현 방법 (Bag of Words, TF-IDF, Word2Vec)의 성능 비교를 했을 때의 결론은 다음과 같습니다:

1. **성능 측면**:
    - **Bag of Words**: 이 방식은 단순히 단어의 발생 빈도를 기반으로 하기 때문에, 문맥적 정보나 단어 간의 관계를 포착하지 못합니다. 따라서 성능이 다른 방법에 비해 낮을 수 있습니다.
    - **TF-IDF**: Bag of Words에 비해 단어의 중요도를 고려하므로 성능 향상을 기대할 수 있습니다. 그러나 여전히 문맥 정보를 포착하지 못하는 단점이 있습니다.
    - **Word2Vec**: 단어의 문맥적 정보와 의미를 잘 반영하여 벡터화하기 때문에, 보다 정확한 감성 분석 결과를 얻을 수 있습니다. 그러나 충분한 학습 데이터가 필요하며, 학습 시간이 길어질 수 있습니다.
2. **기능 측면**:
    - **Bag of Words와 TF-IDF**: 이 두 방법은 단어의 순서나 문맥 정보를 포착하지 못합니다. 그러나 구현이 간단하고 학습 속도가 빠르다는 장점이 있습니다.
    - **Word2Vec**: 단어 간의 유사성이나 관계를 포착할 수 있기 때문에, 단순한 감성 분석 외에도 다양한 NLP 작업에 활용할 수 있습니다.